In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import json
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)
print("✓ Libraries imported successfully")

## Step 1: Load Drug Profiles with Tolerance Data

Load substances that have documented tolerance models.

Expected tolerance parameters:
- **Neuro bucket weights**: How much each mechanism contributes to tolerance
- **Tolerance gain rate**: How quickly tolerance builds (0.5 = slow, 1.0 = normal, 2.0 = fast)
- **Tolerance decay days**: Time for tolerance to reset (7-90 days typical)
- **Cross-tolerance patterns**: Which substances share tolerance

In [ ]:
def load_tolerance_data(data_path='../../data_collector/processed/tolerance_models.json'):
    """
    Load documented tolerance models.
    
    Expected structure:
    {
        "MDMA": {
            "neuro_buckets": {
                "serotonin_release": {"weight": 0.9},
                "dopamine_release": {"weight": 0.3},
                "stimulant": {"weight": 0.6}
            },
            "tolerance_gain_rate": 1.0,
            "tolerance_decay_days": 90,
            "half_life_hours": 8,
            "duration_hours": 6,
            "categories": ["stimulant", "entactogen"],
            "model_origin": "manual",
            "confidence": "High"
        }
    }
    """
    try:
        with open(data_path, 'r') as f:
            tolerance_data = json.load(f)
        print(f"✓ Loaded {len(tolerance_data)} tolerance models")
        return tolerance_data
    except FileNotFoundError:
        print(f"⚠ File not found: {data_path}")
        print("Creating synthetic sample tolerance data...")
        return create_sample_tolerance_data()

def create_sample_tolerance_data():
    """Create sample tolerance models for demonstration"""
    return {
        "MDMA": {
            "neuro_buckets": {
                "serotonin_release": {"weight": 0.9},
                "dopamine_release": {"weight": 0.3},
                "stimulant": {"weight": 0.6}
            },
            "tolerance_gain_rate": 1.0,
            "tolerance_decay_days": 90,
            "half_life_hours": 8,
            "duration_hours": 6,
            "categories": ["stimulant", "entactogen"],
            "model_origin": "manual",
            "confidence": "High"
        },
        "LSD": {
            "neuro_buckets": {
                "serotonin_agonist": {"weight": 0.95},
                "dopamine_modulation": {"weight": 0.2}
            },
            "tolerance_gain_rate": 2.0,
            "tolerance_decay_days": 14,
            "half_life_hours": 5,
            "duration_hours": 12,
            "categories": ["psychedelic"],
            "model_origin": "manual",
            "confidence": "High"
        },
        "Psilocybin": {
            "neuro_buckets": {
                "serotonin_agonist": {"weight": 0.9}
            },
            "tolerance_gain_rate": 2.0,
            "tolerance_decay_days": 7,
            "half_life_hours": 3,
            "duration_hours": 6,
            "categories": ["psychedelic"],
            "model_origin": "manual",
            "confidence": "High"
        },
        "Amphetamine": {
            "neuro_buckets": {
                "dopamine_release": {"weight": 0.8},
                "stimulant": {"weight": 0.9}
            },
            "tolerance_gain_rate": 1.5,
            "tolerance_decay_days": 21,
            "half_life_hours": 10,
            "duration_hours": 8,
            "categories": ["stimulant"],
            "model_origin": "manual",
            "confidence": "High"
        },
        "Cocaine": {
            "neuro_buckets": {
                "dopamine_reuptake": {"weight": 0.9},
                "stimulant": {"weight": 0.95}
            },
            "tolerance_gain_rate": 1.2,
            "tolerance_decay_days": 14,
            "half_life_hours": 1,
            "duration_hours": 2,
            "categories": ["stimulant"],
            "model_origin": "manual",
            "confidence": "High"
        }
    }

# Load tolerance data
tolerance_data = load_tolerance_data()
print(f"\nSubstances with tolerance models: {list(tolerance_data.keys())}")

## Step 2: Extract Tolerance-Relevant Features

Extract features that correlate with tolerance development patterns.

In [ ]:
def extract_tolerance_features(substance_data):
    """
    Extract features relevant to tolerance modeling.
    
    Returns both:
    - Input features (X): mechanism profile, pharmacokinetics
    - Target values (y): tolerance parameters to predict
    """
    features = {}
    targets = {}
    
    # Neurotransmitter bucket weights (input features)
    neuro_buckets = substance_data.get('neuro_buckets', {})
    for bucket in ['serotonin_release', 'serotonin_agonist', 'dopamine_release', 
                   'dopamine_reuptake', 'dopamine_modulation', 'gaba_positive', 
                   'nmda_antagonist', 'stimulant', 'cb1_agonist', 'opioid']:
        weight = neuro_buckets.get(bucket, {}).get('weight', 0.0) if isinstance(neuro_buckets.get(bucket), dict) else neuro_buckets.get(bucket, 0.0)
        features[f'neuro_{bucket}'] = weight
    
    # Pharmacokinetics (input features)
    features['half_life_hours'] = substance_data.get('half_life_hours', 0)
    features['duration_hours'] = substance_data.get('duration_hours', 0)
    
    # Category indicators (input features)
    categories = substance_data.get('categories', [])
    for cat in ['stimulant', 'depressant', 'psychedelic', 'entactogen', 'dissociative']:
        features[f'cat_{cat}'] = 1 if cat in categories else 0
    
    # Tolerance parameters (target values to predict)
    targets['tolerance_gain_rate'] = substance_data.get('tolerance_gain_rate', 1.0)
    targets['tolerance_decay_days'] = substance_data.get('tolerance_decay_days', 28)
    
    return features, targets

# Extract features for all substances with tolerance models
X_data = []
y_gain_rate = []
y_decay_days = []
substance_names = []

for substance_name, data in tolerance_data.items():
    features, targets = extract_tolerance_features(data)
    X_data.append(features)
    y_gain_rate.append(targets['tolerance_gain_rate'])
    y_decay_days.append(targets['tolerance_decay_days'])
    substance_names.append(substance_name)

X_df = pd.DataFrame(X_data)
y_gain_rate = np.array(y_gain_rate)
y_decay_days = np.array(y_decay_days)

print(f"✓ Extracted features for {len(X_df)} substances")
print(f"Feature dimensions: {X_df.shape}")
print(f"\nTarget statistics:")
print(f"  Gain rate: min={y_gain_rate.min():.2f}, max={y_gain_rate.max():.2f}, mean={y_gain_rate.mean():.2f}")
print(f"  Decay days: min={y_decay_days.min():.0f}, max={y_decay_days.max():.0f}, mean={y_decay_days.mean():.0f}")
print("\nSample data:")
X_df.head()

## Step 3: Train K-Nearest Neighbors Regressor

Use KNN to predict tolerance parameters based on mechanism similarity to known substances.

KNN is ideal here because:
- Interpretable: "similar to X, Y, Z"
- Naturally handles mechanism similarity
- Conservative: averages nearby known values

In [ ]:
from sklearn.model_selection import cross_val_score

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_df)

# Train KNN for tolerance gain rate
knn_gain = KNeighborsRegressor(n_neighbors=3, weights='distance')
knn_gain.fit(X_scaled, y_gain_rate)

# Train KNN for tolerance decay days
knn_decay = KNeighborsRegressor(n_neighbors=3, weights='distance')
knn_decay.fit(X_scaled, y_decay_days)

# Cross-validation scores
cv_gain = cross_val_score(knn_gain, X_scaled, y_gain_rate, cv=3, scoring='r2')
cv_decay = cross_val_score(knn_decay, X_scaled, y_decay_days, cv=3, scoring='r2')

print("✓ Models trained successfully")
print(f"\nCross-validation R² scores:")
print(f"  Gain rate: {cv_gain.mean():.3f} ± {cv_gain.std():.3f}")
print(f"  Decay days: {cv_decay.mean():.3f} ± {cv_decay.std():.3f}")

## Step 4: Implement Tolerance Inference Function

Create a function that estimates tolerance parameters for substances without documented models.

In [ ]:
def estimate_tolerance_model(substance_profile, 
                            tolerance_data, 
                            scaler, 
                            knn_gain, 
                            knn_decay,
                            X_df,
                            substance_names):
    """
    Estimate tolerance model for a substance using ML inference.
    
    Returns:
    {
        "model_origin": "ml_inferred",
        "confidence": str,
        "confidence_score": float,
        "derived_from": list[str],
        "neuro_buckets": dict,
        "tolerance_gain_rate": float,
        "tolerance_decay_days": int,
        "notes": str
    }
    """
    # Extract features from substance profile
    features, _ = extract_tolerance_features(substance_profile)
    X_new = pd.DataFrame([features])
    
    # Ensure all columns match training data
    for col in X_df.columns:
        if col not in X_new.columns:
            X_new[col] = 0
    X_new = X_new[X_df.columns]
    
    # Scale features
    X_new_scaled = scaler.transform(X_new)
    
    # Predict tolerance parameters
    gain_rate_pred = knn_gain.predict(X_new_scaled)[0]
    decay_days_pred = knn_decay.predict(X_new_scaled)[0]
    
    # Find nearest neighbors for explanation
    distances, indices = knn_gain.kneighbors(X_new_scaled, n_neighbors=3)
    similar_substances = [substance_names[idx] for idx in indices[0]]
    avg_distance = distances[0].mean()
    
    # Compute confidence based on distance to nearest neighbors
    # Lower distance = higher confidence
    confidence_score = 1.0 / (1.0 + avg_distance)
    
    if confidence_score >= 0.7:
        confidence = "Medium"  # Never "High" for ML-inferred
    elif confidence_score >= 0.5:
        confidence = "Medium-Low"
    else:
        confidence = "Low"
    
    # Apply conservative adjustments
    # Slightly increase decay time for uncertainty
    decay_days_pred = int(decay_days_pred * 1.1)
    
    # Extract neuro bucket weights from substance profile
    neuro_buckets = substance_profile.get('neuro_buckets', {})
    
    # Generate notes
    notes = f"Estimated via pharmacological similarity to {', '.join(similar_substances)}. "
    if confidence_score < 0.6:
        notes += "Conservative decay applied due to limited mechanistic similarity. "
    notes += "This is a guideline-level estimate, not authoritative data."
    
    return {
        "model_origin": "ml_inferred",
        "confidence": confidence,
        "confidence_score": float(confidence_score),
        "derived_from": similar_substances,
        "neuro_buckets": neuro_buckets,
        "tolerance_gain_rate": float(round(gain_rate_pred, 2)),
        "tolerance_decay_days": int(decay_days_pred),
        "notes": notes
    }

print("✓ Tolerance estimation function ready")

## Step 5: Test Tolerance Estimation

Test the model on substances with and without known tolerance data.

In [ ]:
# Test on known substances (should be similar to documented values)
print("=" * 60)
print("Testing on KNOWN substances (validation):")
print("=" * 60)

test_substances = ["MDMA", "LSD"]

for substance_name in test_substances:
    documented = tolerance_data[substance_name]
    
    print(f"\n{substance_name}")
    print(f"  Documented:")
    print(f"    Gain rate: {documented['tolerance_gain_rate']}")
    print(f"    Decay days: {documented['tolerance_decay_days']}")
    
    # Estimate using ML (simulating unknown substance)
    estimated = estimate_tolerance_model(
        documented, tolerance_data, scaler, 
        knn_gain, knn_decay, X_df, substance_names
    )
    
    print(f"  ML Estimate:")
    print(f"    Gain rate: {estimated['tolerance_gain_rate']}")
    print(f"    Decay days: {estimated['tolerance_decay_days']}")
    print(f"    Confidence: {estimated['confidence']} ({estimated['confidence_score']:.2f})")
    print(f"    Derived from: {', '.join(estimated['derived_from'])}")

# Test on hypothetical new substances
print("\n" + "=" * 60)
print("Testing on HYPOTHETICAL new substances:")
print("=" * 60)

# Example: Novel serotonergic entactogen
novel_entactogen = {
    "neuro_buckets": {
        "serotonin_release": {"weight": 0.85},
        "dopamine_release": {"weight": 0.4},
        "stimulant": {"weight": 0.5}
    },
    "half_life_hours": 6,
    "duration_hours": 5,
    "categories": ["entactogen", "stimulant"]
}

print("\nNovel Serotonergic Entactogen (MDMA-like)")
estimated = estimate_tolerance_model(
    novel_entactogen, tolerance_data, scaler,
    knn_gain, knn_decay, X_df, substance_names
)
print(f"  Estimated Gain rate: {estimated['tolerance_gain_rate']}")
print(f"  Estimated Decay days: {estimated['tolerance_decay_days']}")
print(f"  Confidence: {estimated['confidence']} ({estimated['confidence_score']:.2f})")
print(f"  Derived from: {', '.join(estimated['derived_from'])}")
print(f"  Notes: {estimated['notes']}")

# Example: Novel psychedelic
novel_psychedelic = {
    "neuro_buckets": {
        "serotonin_agonist": {"weight": 0.92}
    },
    "half_life_hours": 4,
    "duration_hours": 8,
    "categories": ["psychedelic"]
}

print("\nNovel 5-HT2A Agonist Psychedelic")
estimated = estimate_tolerance_model(
    novel_psychedelic, tolerance_data, scaler,
    knn_gain, knn_decay, X_df, substance_names
)
print(f"  Estimated Gain rate: {estimated['tolerance_gain_rate']}")
print(f"  Estimated Decay days: {estimated['tolerance_decay_days']}")
print(f"  Confidence: {estimated['confidence']} ({estimated['confidence_score']:.2f})")
print(f"  Derived from: {', '.join(estimated['derived_from'])}")
print(f"  Notes: {estimated['notes']}")

## Step 6: Visualize Feature Importance

Understand which mechanisms most strongly correlate with tolerance parameters.

In [ ]:
# Train Random Forest for feature importance analysis
rf_gain = RandomForestRegressor(n_estimators=100, random_state=42)
rf_gain.fit(X_scaled, y_gain_rate)

rf_decay = RandomForestRegressor(n_estimators=100, random_state=42)
rf_decay.fit(X_scaled, y_decay_days)

# Create feature importance dataframes
importance_gain = pd.DataFrame({
    'feature': X_df.columns,
    'importance': rf_gain.feature_importances_
}).sort_values('importance', ascending=False)

importance_decay = pd.DataFrame({
    'feature': X_df.columns,
    'importance': rf_decay.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 10 features for Tolerance Gain Rate:")
print(importance_gain.head(10).to_string(index=False))

print("\n\nTop 10 features for Tolerance Decay Days:")
print(importance_decay.head(10).to_string(index=False))

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

importance_gain.head(10).plot(x='feature', y='importance', kind='barh', ax=ax1, legend=False)
ax1.set_title('Feature Importance: Tolerance Gain Rate')
ax1.set_xlabel('Importance')

importance_decay.head(10).plot(x='feature', y='importance', kind='barh', ax=ax2, legend=False)
ax2.set_title('Feature Importance: Tolerance Decay Days')
ax2.set_xlabel('Importance')

plt.tight_layout()
plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
print("\n✓ Feature importance plot saved to feature_importance.png")

## Step 7: Save Models for API Integration

Export trained models and preprocessing objects.

In [ ]:
import pickle

# Create models directory
models_dir = Path('models')
models_dir.mkdir(exist_ok=True)

# Save models
with open(models_dir / 'tolerance_gain_knn.pkl', 'wb') as f:
    pickle.dump(knn_gain, f)

with open(models_dir / 'tolerance_decay_knn.pkl', 'wb') as f:
    pickle.dump(knn_decay, f)

with open(models_dir / 'tolerance_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("✓ Models saved:")
print(f"  - {models_dir / 'tolerance_gain_knn.pkl'}")
print(f"  - {models_dir / 'tolerance_decay_knn.pkl'}")
print(f"  - {models_dir / 'tolerance_scaler.pkl'}")

# Save feature columns and substance names for reference
metadata = {
    'feature_columns': X_df.columns.tolist(),
    'substance_names': substance_names,
    'training_substances': {
        name: {
            'gain_rate': float(tolerance_data[name]['tolerance_gain_rate']),
            'decay_days': int(tolerance_data[name]['tolerance_decay_days'])
        }
        for name in substance_names
    }
}

with open(models_dir / 'tolerance_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"  - {models_dir / 'tolerance_metadata.json'}")
print("\n✓ Model export complete. Ready for API integration.")

## Summary

This notebook implements ML-assisted tolerance model inference:

✅ **Estimates tolerance parameters** for under-documented substances  
✅ **Based on mechanism similarity** to well-studied drugs  
✅ **Conservative by design** - applies safety margins to predictions  
✅ **Never overrides manual models** - only fills gaps  
✅ **Provides derivation metadata** for transparency  

### Key Outputs

The model predicts:
- `tolerance_gain_rate`: How quickly tolerance develops (0.5-2.0)
- `tolerance_decay_days`: Time for tolerance reset (7-90+ days)
- `confidence`: Reliability of estimate (Low/Medium)
- `derived_from`: Similar substances used for inference

### Integration Points

1. `tolerance_service.py` - API service layer
2. Flutter app - Tolerance calculator logic
3. Database - Store inferred models with metadata

**Important**: ML-inferred models are labeled as estimates and never presented as authoritative data.